In [1]:
!pip install git+https://github.com/ericsuh/dirichlet.git

  Cloning https://github.com/ericsuh/dirichlet.git to /tmp/pip-req-build-4nivgtwn
  Running command git clone -q https://github.com/ericsuh/dirichlet.git /tmp/pip-req-build-4nivgtwn
  Created wheel for dirichlet: filename=dirichlet-0.9-cp36-none-any.whl size=7336 sha256=2012c0143fe37eb95c1ce0cfd0652cf9891e09f442b49cd32bec20053cecaf62
  Stored in directory: /tmp/pip-ephem-wheel-cache-82ah97gl/wheels/52/43/bd/c80dda99ca3b3e39173dc368b70a9a79950d81908d43db02bc
Successfully built dirichlet


In [2]:
import numpy as np
import dirichlet
import torch.nn as nn
import torch.nn.functional as F

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

from matplotlib import pyplot as plt
from Models import Classification_Module as Classification_Module
from Models import Focus_Module as Focus_Module

import copy

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")
a0 = np.array([1,2,100])
D0 = np.random.dirichlet(a0,30000)
dirichlet.mle(D0,tol=1e-3,)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda
cuda


In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)


testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=10, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=10, shuffle=False)


classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

foreground_classes = {'plane', 'car', 'bird'}

background_classes = {'cat', 'deer', 'dog', 'frog', 'horse','ship', 'truck'}

# print(type(foreground_classes))

In [5]:
dataiter = iter(trainloader)
background_data=[]
background_label=[]
foreground_data=[]
foreground_label=[]
batch_size=10

for i in range(5000):   #5000*batch_size = 50000 data points
  images, labels = dataiter.next()
  for j in range(batch_size):
    if(classes[labels[j]] in background_classes):
      img = images[j].tolist()
      background_data.append(img)
      background_label.append(labels[j])
    else:
      img = images[j].tolist()
      foreground_data.append(img)
      foreground_label.append(labels[j])
            
foreground_data = torch.tensor(foreground_data)
foreground_label = torch.tensor(foreground_label)
background_data = torch.tensor(background_data)
background_label = torch.tensor(background_label)

In [6]:
def create_mosaic_img(bg_idx,fg_idx,fg): 
  """
  bg_idx : list of indexes of background_data[] to be used as background images in mosaic
  fg_idx : index of image to be used as foreground image from foreground data
  fg : at what position/index foreground image has to be stored out of 0-8
  """
  image_list=[]
  j=0
  for i in range(9):
    if i != fg:
      image_list.append(background_data[bg_idx[j]])
      j+=1
    else: 
      image_list.append(foreground_data[fg_idx])
      label = foreground_label[fg_idx]  #-7  # minus 7 because our fore ground classes are 7,8,9 but we have to store it as 0,1,2
  #image_list = np.concatenate(image_list ,axis=0)
  image_list = torch.stack(image_list) 
  return image_list,label

In [7]:
desired_num = 30000
mosaic_list_of_images =[]      # list of mosaic images, each mosaic image is saved as list of 9 images
fore_idx =[]                   # list of indexes at which foreground image is present in a mosaic image i.e from 0 to 9               
mosaic_label=[]                # label of mosaic image = foreground class present in that mosaic
for i in range(desired_num):
  np.random.seed(i)
  bg_idx = np.random.randint(0,35000,8)
  fg_idx = np.random.randint(0,15000)
  fg = np.random.randint(0,1)
  fore_idx.append(fg)
  image_list,label = create_mosaic_img(bg_idx,fg_idx,fg)
  mosaic_list_of_images.append(image_list)
  mosaic_label.append(label)

In [8]:
class MosaicDataset(Dataset):
  """MosaicDataset dataset."""

  def __init__(self, mosaic_list_of_images, mosaic_label, fore_idx):
    """
      Args:
        csv_file (string): Path to the csv file with annotations.
        root_dir (string): Directory with all the images.
        transform (callable, optional): Optional transform to be applied
            on a sample.
    """
    self.mosaic = mosaic_list_of_images
    self.label = mosaic_label
    self.fore_idx = fore_idx

  def __len__(self):
    return len(self.label)

  def __getitem__(self, idx):
    return self.mosaic[idx] , self.label[idx], self.fore_idx[idx]

batch = 250
msd = MosaicDataset(mosaic_list_of_images, mosaic_label , fore_idx)
train_loader = DataLoader( msd,batch_size= batch ,shuffle=True)

In [9]:
test_images = []
fore_idx_test =[]                   #list of indexes at which foreground image is present in a mosaic image                
test_label=[]                # label of mosaic image = foreground class present in that mosaic
for i in range(10000):
  np.random.seed(i+30000)
  bg_idx = np.random.randint(0,35000,8)
  fg_idx = np.random.randint(0,15000)
  fg = np.random.randint(0,1)
  fore_idx_test.append(fg)
  image_list,label = create_mosaic_img(bg_idx,fg_idx,fg)
  test_images.append(image_list)
  test_label.append(label)
test_data = MosaicDataset(test_images,test_label,fore_idx_test)
test_loader = DataLoader( test_data,batch_size= batch ,shuffle=False)

In [10]:
focus_net =  Focus_Module(3,1).double()
focus_net = focus_net.to(device)

In [11]:
classify  = Classification_Module(3,3).double()
classify = classify.to(device)
classify

Classification_Module(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 8, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=200, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
  (fc4): Linear(in_features=10, out_features=3, bias=True)
)

In [12]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer_focus = optim.SGD(focus_net.parameters(), lr=0.01, momentum=0.9)
optimizer_classify = optim.SGD(classify.parameters(), lr=0.01, momentum=0.9)

In [13]:
train_dirich_param = []
test_dirich_param = []

In [14]:
tr_loss = []

for epoch in range(150):  # loop over the dataset multiple times
    running_loss = 0.0
    cnt=0
    iteration = 30000 // batch
    ep_loss = []
    train_alpha = []
    test_alpha = []
    focus_net.train()
    classify.train()

    for i, data in  enumerate(train_loader):
        inputs , labels , fgrnd_idx = data
        inputs = inputs.double()
        inputs,labels = inputs.to("cuda"),labels.to("cuda")
        optimizer_focus.zero_grad()
        optimizer_classify.zero_grad()
        avg_data , alphas = focus_net(inputs)
        outputs = classify(avg_data)
        _, predicted = torch.max(outputs.data, 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_focus.step()
        optimizer_classify.step()

        running_loss += loss.item()
        mini = 40
        if cnt % mini == mini-1:    # print every mini mini-batches
            print('[%d, %5d] loss: %.3f' %(epoch + 1, cnt + 1, running_loss / mini))
            ep_loss.append(running_loss/mini)
            running_loss = 0.0  
        cnt=cnt+1
        if epoch %5 == 0:
          train_alpha.append(alphas.detach().cpu().numpy())
    
    if epoch %5 == 0 :
      focus_net.eval()
      classify.eval()
      #trunning_loss  = 0
      for i,data in enumerate(test_loader):
        tinput,tlabels,frgnd_idx = data
        tinput = tinput.double()
        tinput,tlabels =  tinput.to(device),tlabels.to(device)
        tavg_data,talpha = focus_net(tinput)
        #toutput = classify(tavg_data)
        test_alpha.append(talpha.detach().cpu().numpy())
        #toutput = classify(avg_data)
        tr_loss.append(np.mean(ep_loss))
      train_alpha = np.concatenate(train_alpha,axis=0)  
      test_alpha = np.concatenate(test_alpha,axis=0)
      print(train_alpha.shape,test_alpha.shape )
      trparam = dirichlet.mle(train_alpha,method = 'fixedpoint',tol=1e-1)
      tsparam = dirichlet.mle(test_alpha,method ='fixedpoint',tol=0.5)
      print("training dirichlet parameter",trparam)
      print("test dirichlet parameter",tsparam)
      train_dirich_param.append(trparam) 
      test_dirich_param.append(tsparam)

#train_dirich_param = np.concatenate(train_dirich_param,axis=0)
#test_dirich_param = np.concatenate(test_dirich_param,axis=0)   
print('Finished Training')

[1,    40] loss: 1.101
[1,    80] loss: 1.099
[1,   120] loss: 1.099
(30000, 9) (10000, 9)
training dirichlet parameter [10122223.37058559 10122244.43798501 10122275.30198231 10122239.77259436
 10122267.59867718 10122269.22238208 10122227.60369902 10122240.62736123
 10122234.31967872]
test dirichlet parameter [9983559.93882043 9983622.17643392 9983608.02055493 9983584.16986548
 9983634.92785795 9983614.29293573 9983586.23489376 9983571.70089145
 9983575.21062284]
[2,    40] loss: 1.099
[2,    80] loss: 1.099
[2,   120] loss: 1.099
[3,    40] loss: 1.099
[3,    80] loss: 1.099
[3,   120] loss: 1.098
[4,    40] loss: 1.098
[4,    80] loss: 1.098
[4,   120] loss: 1.098
[5,    40] loss: 1.098
[5,    80] loss: 1.098
[5,   120] loss: 1.098
[6,    40] loss: 1.098
[6,    80] loss: 1.098
[6,   120] loss: 1.097
(30000, 9) (10000, 9)
training dirichlet parameter [10132887.14836804 10132906.12970989 10132937.12619784 10132901.55116998
 10132929.40075678 10132930.94107212 10132889.34724169 10132902

In [22]:
df = pd.DataFrame(np.array(train_dirich_param))

In [25]:
df.to_csv("train_dirich_params.csv",header=False)

In [26]:
df = pd.DataFrame(np.array(test_dirich_param))
df.to_csv("test_dirich_params.csv",header=False)